In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
import pandas

train_ud = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\train-ud_paths_v0.0.5.csv').set_index(['id'])
test_ud = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\test-ud_paths_v0.0.5.csv').set_index(['id'])

train_ucca = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\train-ucca_paths_v0.0.5.csv').set_index(['id'])
test_ucca = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\test-ucca_paths_v0.0.5.csv').set_index(['id'])

In [3]:
train_ner = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\train-ud_paths_with_ner_v0.0.6.csv').set_index(['id'])
test_ner = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\test-ud_paths_with_ner_v0.0.6.csv').set_index(['id'])

train_ids_with_proper_ner = train_ner[(train_ner['type1']==train_ner['type1_corenlp']) & (train_ner['type2']==train_ner['type2_corenlp'])].drop(['docid', 'tokens', 'relation', 'path', 'lemmas_on_path', 'type1', 'type2', 'ent1_head', 'ent2_head', 'type1_corenlp', 'type2_corenlp'], axis=1)
test_ids_with_proper_ner = test_ner[(test_ner['type1']==test_ner['type1_corenlp']) & (test_ner['type2']==test_ner['type2_corenlp'])].drop(['docid', 'tokens', 'relation', 'path', 'lemmas_on_path', 'type1', 'type2', 'ent1_head', 'ent2_head', 'type1_corenlp', 'type2_corenlp'], axis=1)

In [4]:
train_ucca = train_ids_with_proper_ner.join(train_ucca).dropna(axis=0).reset_index()
test_ucca = test_ids_with_proper_ner.join(test_ucca).dropna(axis=0).reset_index()


train_ud = train_ids_with_proper_ner.join(train_ud).reset_index()
test_ud = test_ids_with_proper_ner.join(test_ud).reset_index()

In [5]:
def transform_row(r):
    return '{entity1} {org_path} {entity2}'.format(entity1=r.type1[0:3], org_path=r.path, entity2=r.type2[0:3])

train_ud['path'] = train_ud.apply(transform_row, axis=1)
test_ud['path'] = test_ud.apply(transform_row, axis=1)

train_ucca['path'] = train_ucca.apply(transform_row, axis=1)
test_ucca['path'] = test_ucca.apply(transform_row, axis=1)

In [6]:
train_ud = train_ud.rename(columns={'path' : 'ud_path'})
test_ud = test_ud.rename(columns={'path' : 'ud_path'})

train_ucca = train_ucca.rename(columns={'path' : 'ucca_path'})
test_ucca = test_ucca.rename(columns={'path' : 'ucca_path'})

In [7]:
test = pandas.merge(test_ud, test_ucca[['id', 'ucca_path']], on='id').dropna()
test = test[test['relation'] != 'no_relation']

In [8]:
train_ud_total_by_r_and_p = train_ud[train_ud['relation'] != 'no_relation'].groupby(['relation','ud_path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['relation','total'], ascending=[True,False]).reset_index(drop=True)
train_ucca_total_by_r_and_p = train_ucca[train_ucca['relation'] != 'no_relation'].groupby(['relation','ucca_path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['relation','total'], ascending=[True,False]).reset_index(drop=True)

In [9]:
# we're going to remove paths that are not maximum for their relation

import numpy

path_counts = train_ud[train_ud['relation']!='no_relation'].groupby(['ud_path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)
paths = path_counts['ud_path'].tolist()

relation_counts = train_ud[train_ud['relation']!='no_relation'].groupby(['relation'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)
relations = relation_counts['relation'].tolist()

matrix = numpy.zeros((len(paths), len(relations)), dtype=numpy.int32)

helper = train_ud[train_ud['relation'] != 'no_relation'].groupby(['relation','ud_path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['relation','total'], ascending=[True,False]).set_index(['relation','ud_path'])
path_and_relation_to_count = helper.to_dict()['total']

for path_index, path in enumerate(paths):
    for relation_index, relation in enumerate(relations):
        matrix[path_index,relation_index] = path_and_relation_to_count.get((relation,path), 0)
        
path_to_relation = pandas.DataFrame(data=matrix, index=paths, columns=relations)
path_to_relation['sum'] = path_to_relation.sum(axis=1)
path_to_relation = path_to_relation[path_to_relation['sum'] > 10]


def get_max_for_relation(path, relation):
    try:
        path_to_relation_row = path_to_relation.loc[path]
        return path_to_relation_row.max()
        
    except KeyError:
        return 0


train_ud_total_by_r_and_p['max'] =  train_ud_total_by_r_and_p.apply(lambda row : get_max_for_relation(row.ud_path, row.relation), axis=1)    
#train_ud_total_by_r_and_p = train_ud_total_by_r_and_p[train_ud_total_by_r_and_p['total']==train_ud_total_by_r_and_p['max']]

In [10]:
# we're going to remove paths that are not maximum for their relation
# for UCCA

import numpy

path_counts = train_ucca[train_ucca['relation']!='no_relation'].groupby(['ucca_path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)
paths = path_counts['ucca_path'].tolist()

relation_counts = train_ucca[train_ucca['relation']!='no_relation'].groupby(['relation'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)
relations = relation_counts['relation'].tolist()

matrix = numpy.zeros((len(paths), len(relations)), dtype=numpy.int32)

helper = train_ucca[train_ucca['relation'] != 'no_relation'].groupby(['relation','ucca_path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['relation','total'], ascending=[True,False]).set_index(['relation','ucca_path'])
path_and_relation_to_count = helper.to_dict()['total']

for path_index, path in enumerate(paths):
    for relation_index, relation in enumerate(relations):
        matrix[path_index,relation_index] = path_and_relation_to_count.get((relation,path), 0)
        
path_to_relation = pandas.DataFrame(data=matrix, index=paths, columns=relations)
path_to_relation['sum'] = path_to_relation.sum(axis=1)
path_to_relation = path_to_relation[path_to_relation['sum'] > 10]


def get_max_for_relation(path, relation):
    try:
        path_to_relation_row = path_to_relation.loc[path]
        return path_to_relation_row.max()
        
    except KeyError:
        return 0


train_ucca_total_by_r_and_p['max'] =  train_ucca_total_by_r_and_p.apply(lambda row : get_max_for_relation(row.ucca_path, row.relation), axis=1)    
#train_ucca_total_by_r_and_p = train_ucca_total_by_r_and_p[train_ucca_total_by_r_and_p['total']==train_ucca_total_by_r_and_p['max']]

In [11]:
def label_matches(test_row):
    relation = test_row['relation']
    ud_path = test_row['ud_path']
    ucca_path = test_row['ucca_path']
    
    train_ud_row = train_ud_total_by_r_and_p.loc[
        (train_ud_total_by_r_and_p['relation'] == relation) & 
        (train_ud_total_by_r_and_p['ud_path'] == ud_path)]

    train_ucca_row = train_ucca_total_by_r_and_p.loc[
        (train_ucca_total_by_r_and_p['relation'] == relation) & 
        (train_ucca_total_by_r_and_p['ucca_path'] == ucca_path)]


    match_any = not train_ud_row.empty or not train_ucca_row.empty
    match_ud = not train_ud_row.empty
    match_ucca = not  train_ucca_row.empty
    
    return pandas.Series( {'match_any' : match_any, 'match_ud' : match_ud, 'match_ucca' : match_ucca})


matching_results = test.apply( label_matches, axis=1)

In [12]:
test = pandas.concat((test, matching_results), axis=1)

In [13]:
stats = []
relations = train_ud['relation'].unique().tolist()

for relation in relations:
    
    df = test[test['relation']==relation]
    
    matched = df[df['match_any']]['id'].count()
    total = df.shape[0]
    
    stats.append( ('{} ({})'.format(relation,df.shape[0]), round(matched/total,4)) )

matched = test[test['match_any']]['id'].count()
total = test.shape[0]
stats.append( ('overall', round(matched/total,4)) )

stats = sorted(stats, key=lambda x: x[1], reverse=True)

c:\users\jyellin\anaconda3\envs\msc_3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning:

invalid value encountered in long_scalars



In [14]:
import plotly.graph_objs as go

overall_index = next(i for (i, stat) in enumerate(stats) if stat[0] == 'overall')

colors = ['rgb(58,200,225)'] * len(stats)
colors[overall_index] = 'rgb(181,59,89)'

trace = go.Bar(
    x=[stat[0] for stat in stats],
    y=[stat[1] for stat in stats],
    name='Recall',
    width=0.5,
    marker=dict(
        color=colors,
        line=dict(
            color='rgb(8,48,107)',
            width=0.5),
        ),
    opacity=0.6
)

data = [trace]
layout = go.Layout(
    title='UD + UCCA Path - Recall',
    xaxis_title="Relation",
    yaxis_title="Recall",    
    barmode='overlay',
    width=1200,
    xaxis = go.layout.XAxis(
        tickangle = 45,
        automargin = True
        
    )
    
        
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)